In [2]:
import pandas as pd 

In [6]:
mta_hourly_ridership_data_2025 = pd.read_csv("data/MTA_Subway_Hourly_Ridership__Beginning_2025.csv", low_memory=False)

In [7]:
mta_hourly_ridership_data_2025.head()

,transit_timestamp,transit_mode,station_complex_id,station_complex,borough,payment_method,fare_class_category,ridership,transfers,latitude,longitude,Georeference
0,01/03/2025 12:00:00 AM,tram,TRAM2,RI Tramway (Roosevelt),Manhattan,omny,OMNY - Full Fare,12,7,40.757340,-73.95412,POINT (-73.95412 40.75734)
1,01/03/2025 06:00:00 AM,tram,TRAM1,RI Tramway (Manhattan),Manhattan,omny,OMNY - Seniors & Disability,1,1,40.761337,-73.96416,POINT (-73.96416 40.761337)
2,01/03/2025 07:00:00 AM,tram,TRAM2,RI Tramway (Roosevelt),Manhattan,metrocard,Metrocard - Unlimited 30-Day,10,0,40.757340,-73.95412,POINT (-73.95412 40.75734)
3,01/03/2025 08:00:00 AM,tram,TRAM1,RI Tramway (Manhattan),Manhattan,metrocard,Metrocard - Unlimited 30-Day,5,0,40.761337,-73.96416,POINT (-73.96416 40.761337)
4,01/03/2025 10:00:00 AM,tram,TRAM1,RI Tramway (Manhattan),Manhattan,omny,OMNY - Students,1,0,40.761337,-73.96416,POINT (-73.96416 40.761337)


In [8]:
mta_hourly_ridership_data_2025.tail()

,transit_timestamp,transit_mode,station_complex_id,station_complex,borough,payment_method,fare_class_category,ridership,transfers,latitude,longitude,Georeference
7540355,03/24/2025 01:00:00 PM,tram,TRAM1,RI Tramway (Manhattan),Manhattan,metrocard,Metrocard - Seniors & Disability,1,1,40.761337,-73.96416,POINT (-73.96416 40.761337)
7540356,03/24/2025 01:00:00 PM,tram,TRAM2,RI Tramway (Roosevelt),Manhattan,metrocard,Metrocard - Full Fare,17,4,40.757340,-73.95412,POINT (-73.95412 40.75734)
7540357,03/24/2025 04:00:00 PM,tram,TRAM2,RI Tramway (Roosevelt),Manhattan,metrocard,Metrocard - Other,2,0,40.757340,-73.95412,POINT (-73.95412 40.75734)
7540358,03/24/2025 10:00:00 PM,tram,TRAM1,RI Tramway (Manhattan),Manhattan,omny,OMNY - Full Fare,42,11,40.761337,-73.96416,POINT (-73.96416 40.761337)
7540359,03/24/2025 10:00:00 PM,tram,TRAM2,RI Tramway (Roosevelt),Manhattan,metrocard,Metrocard - Fair Fare,1,0,40.757340,-73.95412,POINT (-73.95412 40.75734)


In [9]:
mta_hourly_ridership_data_2025.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7540360 entries, 0 to 7540359
Data columns (total 12 columns):
 #   Column               Dtype  
---  ------               -----  
 0   transit_timestamp    object 
 1   transit_mode         object 
 2   station_complex_id   object 
 3   station_complex      object 
 4   borough              object 
 5   payment_method       object 
 6   fare_class_category  object 
 7   ridership            int64  
 8   transfers            int64  
 9   latitude             float64
 10  longitude            float64
 11  Georeference         object 
dtypes: float64(2), int64(2), object(8)
memory usage: 690.3+ MB


In [ ]:
#remove all boroughs besides "Manhattan" because that's where the traffic congestion law applies
mta_hourly_ridership_data_2025 = mta_hourly_ridership_data_2025[mta_hourly_ridership_data_2025['borough'] == 'Manhattan']

In [13]:
#remove payment_method and fare_class_category columns
mta_hourly_ridership_data_2025 = mta_hourly_ridership_data_2025.drop(columns=['payment_method', 'fare_class_category'])

In [14]:
mta_hourly_ridership_data_2025.head()

,transit_timestamp,transit_mode,station_complex_id,station_complex,borough,ridership,transfers,latitude,longitude,Georeference
0,01/03/2025 12:00:00 AM,tram,TRAM2,RI Tramway (Roosevelt),Manhattan,12,7,40.757340,-73.95412,POINT (-73.95412 40.75734)
1,01/03/2025 06:00:00 AM,tram,TRAM1,RI Tramway (Manhattan),Manhattan,1,1,40.761337,-73.96416,POINT (-73.96416 40.761337)
2,01/03/2025 07:00:00 AM,tram,TRAM2,RI Tramway (Roosevelt),Manhattan,10,0,40.757340,-73.95412,POINT (-73.95412 40.75734)
3,01/03/2025 08:00:00 AM,tram,TRAM1,RI Tramway (Manhattan),Manhattan,5,0,40.761337,-73.96416,POINT (-73.96416 40.761337)
4,01/03/2025 10:00:00 AM,tram,TRAM1,RI Tramway (Manhattan),Manhattan,1,0,40.761337,-73.96416,POINT (-73.96416 40.761337)


In order to count ridership in and out of the traffic congestion region, we have to load this data into ArcGIS Pro, run XY Table-to-Point, and then add a field for stations south of 60th Street. This'll be a dummy variable, where 1 = south of 60th Street and 0 = north of 60th Street. We can then use the Select by Attributes tool to select all the stations south of 60th Street, and then export that selection to a new shapefile. This will give us a shapefile with only the stations south of 60th Street, which we can then use to count ridership in and out of the traffic congestion region.
